In [1]:
%pip install pandas
%pip install sklearn
%pip install simpletransformers
%pip install torch torchvision torchaudio
%pip install torchtext

     |████████████████████████████████| 11.1 MB 977 kB/s eta 0:00:01
  Using cached numpy-1.22.3-cp310-cp310-macosx_10_14_x86_64.whl (17.6 MB)
  Using cached pytz-2022.1-py2.py3-none-any.whl (503 kB)
Note: you may need to restart the kernel to use updated packages.
  Using cached scikit_learn-1.0.2-cp310-cp310-macosx_10_13_x86_64.whl (8.0 MB)
  Using cached threadpoolctl-3.1.0-py3-none-any.whl (14 kB)
  Using cached joblib-1.1.0-py2.py3-none-any.whl (306 kB)
  Using cached scipy-1.8.0-cp310-cp310-macosx_12_0_universal2.macosx_10_9_x86_64.whl (55.6 MB)
  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1310 sha256=474964e72112223bfd62c1d8dd85c8eb9c3008c601b183913d835783d9cd0be0
  Stored in directory: /Users/siddhantshingi/Library/Caches/pip/wheels/9b/13/01/6f3a7fd641f90e1f6c8c7cded057f3394f451f340371c68f3d
Successfully built sklearn
Note: you may need to restart the kernel to use updated packages.
     |████████████████████████████████| 249 kB 4.3 MB/s eta 0:00:

In [35]:
import datasets
train_data, test_data = datasets.load_dataset('imdb', split =['train', 'test'], 
                                             cache_dir='./data/')

Reusing dataset imdb (./data/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1)


  0%|          | 0/2 [00:00<?, ?it/s]

In [36]:
train_data_dev, test_data_dev = train_data.select([0, 10, 20, 30, 40, 50]), test_data.select([0, 10, 20, 30, 40, 50])

In [74]:
from transformers import BertTokenizer, BertModel
import torch
import numpy as np
device = torch.device('cpu')

In [ ]:
def get_encoder():
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    model = BertModel.from_pretrained('bert-base-uncased')

    return model, tokenizer

In [3]:
tokenizer, model = get_encoder()

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [64]:
def tokenization(batched_text):
    return tokenizer(batched_text['text'], padding = 'max_length', truncation=True, max_length = 512)
train_data_dev = train_data_dev.map(tokenization, batched = True, batch_size = len(train_data_dev))
test_data_dev = test_data_dev.map(tokenization, batched = True, batch_size = len(test_data_dev))

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [75]:
ids = torch.tensor(train_data_dev['input_ids']).to(device)
mask = torch.tensor(train_data_dev['attention_mask']).to(device)

output = model(ids, mask)

In [76]:
final_layer = output.last_hidden_state

In [77]:
final_layer[:,0,:].size()

torch.Size([6, 768])

AttributeError: 'Dataset' object has no attribute 'to'

In [78]:
train_data_dev

Dataset({
    features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 6
})

In [80]:
from datasets import Dataset
from typing import Optional, Tuple, NamedTuple, List

In [84]:
class Augmentor():
    """Base class for text augmentors."""
    def __init__(self):
        pass

    def augment(self, x: Dataset) -> Dataset:
        raise NotImplementedError(f"Dataset.augment should be implemented.")

    def __call__(
            self, x: Dataset,
    ) -> Tuple[torch.Tensor, torch.Tensor]:
        return self.augment(x)


In [85]:
class Identity(Augmentor):
    def __init__(self):
        super(Identity, self).__init__()

    def augment(self, x: Dataset) -> Dataset:
        return x


In [86]:
aug = Identity()
aug(train_data)

Dataset({
    features: ['text', 'label'],
    num_rows: 25000
})